# Transformaciones Logarítmica y Box-Cox en Columnas Numéricas

## 1. Cargar datos y verificar columnas numéricas

In [1]:

import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_csv("data/fact_laptops_preprocesado.csv")

# Verificar columnas numéricas (excluyendo identificadores)
numeric_cols = df.select_dtypes(include=["float64", "int64"]).drop(columns=["LaptopID"], errors='ignore')
numeric_summary = pd.DataFrame({
    "Columna": numeric_cols.columns,
    "Mínimo": numeric_cols.min(),
    "Tiene valores negativos": (numeric_cols < 0).any(),
    "Tiene ceros": (numeric_cols == 0).any(),
    "Aplicable Log (x+1)": ~(numeric_cols < 0).any(),  # log1p requiere x >= -1
    "Aplicable Box-Cox": ((numeric_cols > 0).all())  # Box-Cox requiere x > 0
}).reset_index(drop=True)

numeric_summary


,Columna,Mínimo,Tiene valores negativos,Tiene ceros,Aplicable Log (x+1),Aplicable Box-Cox
0,Inches,11.60,False,False,True,True
1,Ram,4.00,False,False,True,True
2,Weight,1.00,False,False,True,True
3,PriceEuros,301.23,False,False,True,True
4,CPUFrequency,1.00,False,False,True,True
5,PrimaryStorage,128.00,False,False,True,True
6,SecondaryStorage,0.00,False,True,True,False


## 2. Aplicar Transformación Logarítmica a columnas seleccionadas

In [2]:

# Aplicar log1p (log(x+1)) a columnas con valores no negativos
df['PriceEuros_log'] = np.log1p(df['PriceEuros'])
df['Weight_log'] = np.log1p(df['Weight'])

df[['PriceEuros', 'PriceEuros_log', 'Weight', 'Weight_log']].head()


,PriceEuros,PriceEuros_log,Weight,Weight_log
0,2127.09,7.662980,2.84,1.345472
1,1891.02,7.545400,1.55,0.936093
2,2479.25,7.816115,2.51,1.255616
3,2639.51,7.878727,3.14,1.420696
4,960.99,6.869004,2.34,1.205971


## 3. Aplicar Transformación Box-Cox a columnas seleccionadas

In [3]:

from scipy.stats import boxcox

# Box-Cox solo permite valores estrictamente positivos
# Aplicamos a 'Inches' y 'Ram' si cumplen el criterio
df['Inches_boxcox'], lambda_inches = boxcox(df['Inches'])
df['Ram_boxcox'], lambda_ram = boxcox(df['Ram'])

print(f"Lambda para Inches: {lambda_inches}")
print(f"Lambda para Ram: {lambda_ram}")

df[['Inches', 'Inches_boxcox', 'Ram', 'Ram_boxcox']].head()


Lambda para Inches: 0.7419254973200582
Lambda para Ram: -0.012908228949408108


,Inches,Inches_boxcox,Ram,Ram_boxcox
0,13.0,7.690787,8,2.051781
1,15.7,9.049158,32,3.389357
2,13.6,7.998486,16,2.723561
3,16.0,9.196195,4,1.373964
4,15.1,8.752886,8,2.051781
